In [82]:
from bs4 import BeautifulSoup
import requests
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import time

In [83]:
# this version is more transferable, as compared to the next cell (which relies on path on my computer)
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======


[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Driver [/Users/annasours/.wdm/drivers/chromedriver/mac64/89.0.4389.23/chromedriver] found in cache


In [18]:
# # Setup splinter
# executable_path = {'executable_path': '/Users/annasours/chromedriver/chromedriver'}

# # executable_path = {'executable_path': ChromeDriverManager().install()}
# browser = Browser('chrome', **executable_path, headless=False)

In [3]:
# URLs of pages to be scraped

# NASA Mars News
url_1 = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"
# JPL Mars Space Images - Featured Image
url_2 = "https://www.jpl.nasa.gov/images?search=&category=Mars"
# Mars Facts
url_3 = "https://space-facts.com/mars/"
# Mars Hemispheres
url_4 = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"

### Nasa Mars News

In [84]:
# Try again using broswer
# browser = init_browser()
browser.visit(url_1)

# one way to get the webpage to load slower
time.sleep(2)

In [85]:
html=browser.html
news_soup=BeautifulSoup(html,'html.parser')

first_article=news_soup.find('ul', class_='item_list')
news_title=first_article.find('div', class_='content_title').a.text
news_p=first_article.find('div', class_='article_teaser_body').text

print(news_title)
print('-----'*5)
print(news_p)

NASA's Perseverance Mars Rover Mission Honors Navajo Language
-------------------------
Working with the Navajo Nation, the rover team has named features on Mars with words from the Navajo language.


### JPL Mars Space Image - Featured Image

In [86]:
browser.visit(url_2)

# one way to get the webpage to load slower
time.sleep(2)

In [87]:
dropdown_content=browser.find_by_css('.-active')

for button in dropdown_content:
    button.click()

    
html=browser.html
pic_soup=BeautifulSoup(html,'html.parser')
featured_image_url=pic_soup.find('img', class_='BaseImage')['src']
print(featured_image_url) 

https://d2pn8kiwq2w21t.cloudfront.net/images/jpegPIA24179.2e16d0ba.fill-400x225-c70.jpg


### Mars Facts

In [88]:
import pandas as pd

In [89]:
tables = pd.read_html(url_3)
mars_facts_df=tables[0]
mars_facts_df.columns=['Metric', 'measurement']
mars_facts_df=mars_facts_df.set_index('Metric')
mars_facts_df

,measurement
Metric,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [90]:
html_table = mars_facts_df.to_html()
html_table

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>measurement</th>\n    </tr>\n    <tr>\n      <th>Metric</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr>

In [91]:
# if more needed:
# strip unwanted newlines to clean it up
# html_table.replace('\n', '') 

### Mars Hemispheres

In [92]:
browser.visit(url_4)
time.sleep(2)

In [93]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')


photos = soup.find_all('div', class_='description')

base_url='https://astrogeology.usgs.gov'
img_dict_list=[]


for each_photo in photos:
#     find img_url
    h3 = each_photo.find('h3')
    link = each_photo.find('a')
    href = link['href']
    title=h3.text

    print(f'Getting {title} link')

#     this will click on each photo link.
    browser.links.find_by_partial_text(title).click()
    pic_html = browser.html
    pic_soup = BeautifulSoup(pic_html, 'html.parser')
    source_photo=pic_soup.find('img', class_="wide-image")['src']
    img_url=(base_url+source_photo)

#     create dictionary with title and img url and append to list of dictionaries
    img_dict_list.append({'title': title, 'img_url': img_url})
    
#     go back to original page to follow next link
    browser.visit(url_4)
print('Hemispheres complete')

Getting Cerberus Hemisphere Enhanced link
Getting Schiaparelli Hemisphere Enhanced link
Getting Syrtis Major Hemisphere Enhanced link
Getting Valles Marineris Hemisphere Enhanced link
Hemispheres complete


In [94]:
print(img_dict_list)

[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]
